<a href="https://colab.research.google.com/github/YounSooKimTech/self_study/blob/main/Text_Classification_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df_train = pd.read_excel("/content/BOS_RISK_preprocessed.xlsx")
df_test = pd.read_excel("/content/Service_Risk_preprocessed.xlsx")

df_train.fillna("", inplace=True)
df_test.fillna("", inplace=True)

In [3]:
df_train.head(2)

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,Language,translated,text
0,Algeria,2023,Fluctuation de la monnaies et des prix,Financial and economic instability,"3. External factors (COVID-19, economic, legis...",Highly Possible,Significant Concern,Very High,Monitor,assurer une mise à jour réguliére des données ...,French,Currency and price fluctuations,currency price fluctuation
1,Algeria,2023,Maitrise des processus et turnover,Capacity development,"2. Operational factors (funding, capacity and ...",Highly Possible,High Concern,Very High,Accept,assurer la formation continue du personnel du ...,French,Mastery of processes and turnover,mastery process turnover


In [4]:
df_test.head(2)

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,COMMON_SERVICE_LINE,COMMON_SERVICE_CATEGORY,COMMON_SERVICE,Language,translated,text
0,Afghanistan,2023,A funding cut imposed by the legislative bodies,,,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies,funding cut impose legislative body
1,Afghanistan,2023,A funding cut imposed by the legislative bodies,,,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies,funding cut impose legislative body


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# X_train, y_train
X_train = df_train['text']
X_test = df_test["text"]
y_train = df_train['Standard_risk_L2']

# TF-IDF vectorization (trained by X_train)
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

predicted_labels = rf_classifier.predict(X_test_tfidf)
df_test["Standard_risk_L2"] = predicted_labels


In [8]:
df_test.head(3)

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,COMMON_SERVICE_LINE,COMMON_SERVICE_CATEGORY,COMMON_SERVICE,Language,translated,text
0,Afghanistan,2023,A funding cut imposed by the legislative bodies,Funding,,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies,funding cut impose legislative body
1,Afghanistan,2023,A funding cut imposed by the legislative bodies,Funding,,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies,funding cut impose legislative body
2,Afghanistan,2023,A funding cut imposed by the legislative bodies.,Funding,,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies.,funding cut impose legislative body


In [11]:
# manually correct the label after meetings

# manually correct the labels after discussion

df_test.loc[df_test["RISK_DESCRIPTION"] == "Change in the Security Council mandate of the Mission", "Standard_risk_L2"] = "External factors"
df_test.loc[df_test["RISK_DESCRIPTION"] == "The task might be delayed for weeks.", "Standard_risk_L2"] = "Agencies' commitment and participation"
df_test.loc[df_test["RISK_DESCRIPTION"] == "Low utilisation of the LTA by agencies", "Standard_risk_L2"] = "Underutilization"

df_test.loc[df_test["RISK_DESCRIPTION"] == "Ambulance Services", "Standard_risk_L2"] = "Other"
df_test.loc[df_test["RISK_DESCRIPTION"] == "Human resources to plan and implement is limited. If focused attention is to provided to development and implementation of activities then it is likely that these will delayed or not executed", "Standard_risk_L2"] = "Sourcing personnel (local)"
df_test.loc[df_test["RISK_DESCRIPTION"] == "Ability to meet various types of requirements.", "Standard_risk_L2"] = "Agencies' misalignment"

df_test.loc[df_test["RISK_DESCRIPTION"] == "Preferences  to specific vendors who may not be part of the common LTA thus rendering this service not useful", "Standard_risk_L2"] = "Agencies' commitment and participation"
df_test.loc[df_test["RISK_DESCRIPTION"] == "Ability to meet various types of requirements.", "Standard_risk_L2"] = "Agencies' misalignment"
df_test.loc[df_test["RISK_DESCRIPTION"] == "In-Source: Ability to meet various types of requirements.", "Standard_risk_L2"] = "Agencies' misalignment"

df_test.loc[df_test["RISK_DESCRIPTION"] == "Database not updated", "Standard_risk_L2"] = "Interoperability and data availability"
df_test.loc[df_test["RISK_DESCRIPTION"] == "Difficulties to obtain information at national level", "Standard_risk_L2"] = "Interoperability and data availability"

df_test.loc[df_test["RISK_DESCRIPTION"] == "None", "Standard_risk_L2"] = "No Data"


In [12]:
df_test["Standard_risk_L2"].value_counts()

Agencies' commitment and participation    323
Other                                     207
Supply chain                              119
Agencies' misalignment                     63
Operational capacity                       34
COVID-19                                   27
External factors                           26
Financial and economic instability         23
Funding                                    22
Interoperability and data availability     18
Partnering issues (local)                  16
Security                                   16
Sourcing personnel (local)                 13
No Data                                    12
Underutilization                            8
Operational infrastructure                  6
Banking issues                              4
Capacity development                        4
Local legislation                           3
Operational issues                          1
Name: Standard_risk_L2, dtype: int64

In [15]:

df_test.to_excel('Service_Risk_Labeled.xlsx', index=False)

from google.colab import files
files.download('Service_Risk_Labeled.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>